### cache 装饰器

In [1]:
import functools

In [5]:
import time, datetime

@functools.lru_cache(2)
def sleep(x):
    print(datetime.datetime.now())
    time.sleep(x)
    print(datetime.datetime.now())

In [6]:
sleep(3)

2017-09-24 09:06:17.594932
2017-09-24 09:06:20.597974


In [7]:
sleep(3)

In [8]:
sleep(2)

2017-09-24 09:06:25.991697
2017-09-24 09:06:27.993085


In [9]:
sleep(2)

In [10]:
sleep(1)

2017-09-24 09:06:33.570739
2017-09-24 09:06:34.571038


In [11]:
sleep(3)

2017-09-24 09:06:37.945926
2017-09-24 09:06:40.951043


- 允许过期（通过设置过期时间）
- 没有换出
- 没有清除

- cache 需要具备哪些元素？
- 这些元素需要如何保存
- 通过一种机制，保证同样的输入 能够直接得到缓存，而不是被执行

- 简单的缓存器（只要能够缓存值就可以了）
- 在考虑如何添加过期时间

key -> value

exp = 30 

In [13]:
import functools

key -> 'x=1&y=3'



key -> [(key, value), (key, value)]

In [ ]:
def cache(fn):
    cache = {}
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        # TODO key 如何瓶装
        if key in cache.keys():
            # TODO 超时检测
            return cache[key]
        ret = fn(*args, **kwargs)
        return ret
    return wrap

In [ ]:
@cache
def add(x, y):
    return x + y

In [14]:
import inspect

In [34]:
def add(x, y=3):
    pass

sig = inspect.signature(add)

In [35]:
sig.parameters

mappingproxy({'x': <Parameter "x">, 'y': <Parameter "y=3">})

In [37]:
sig.parameters['y'].default

3

In [20]:
def cache(fn):
    cache = {}
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        key = []
        
        # 处理位置参数
        params = inspect.signature(fn).parameters
        for i, arg in enumerate(args):
            name = list(params.keys())[i]
            key.append((name, arg))
            
        # 关键字参数
        # for k, v in kwargs.items():
        #    key.append((k, v))
            
        key.extend(kwargs.items())
        
        key.sort(key=lambda x: x[0])
        
        key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])
        
        print(key)
        
        
        # TODO key 如何瓶装
        if key in cache.keys():
            # TODO 超时检测
            print('cache hit')
            return cache[key]
        ret = fn(*args, **kwargs)
        cache[key] = ret
        print('cache miss')
        return ret
    return wrap

In [25]:
@cache
def add(x, y=3):
    return x + y

In [28]:
add(3, y=3)

x=3&y=3
cache miss


6

In [24]:
add(2, 2)

x=2&y=2
cache miss


4

In [38]:
def cache(fn):
    cache = {}
    @functools.wraps(fn)
    def wrap(*args, **kwargs):
        key = []
        
        # 对默认值进行 预处理
        # 需要把默认值  和 其参数 存储起来
        # names 中存储的是什么？是我们的参数key
        # 默认值不需要存储，因为 可以通过v.default
        names = set()
        
        # 处理位置参数
        params = inspect.signature(fn).parameters
        for i, arg in enumerate(args):
            name = list(params.keys())[i]
            key.append((name, arg))
            
            names.add(name)
            
        # 关键字参数
        # for k, v in kwargs.items():
        #    key.append((k, v))
            
        key.extend(kwargs.items())
        names.update(kwargs.keys())
        
        # 如果 参数中 有默认值，就需要 往key中添加一对 k-v 对
        for k, v in params.items():
            if k not in names:
                key.append((k, v.default))
        
        key.sort(key=lambda x: x[0])
        
        key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])
        
        print(key)
        
        
        # TODO key 如何瓶装
        if key in cache.keys():
            # TODO 超时检测
            print('cache hit')
            return cache[key]
        ret = fn(*args, **kwargs)
        cache[key] = ret
        print('cache miss')
        return ret
    return wrap

In [39]:
@cache
def add(x, y=3):
    return x + y

In [40]:
add(1)

x=1&y=3
cache miss


4

In [42]:
add(1, 3)

x=1&y=3
cache hit


4

In [43]:
d = {('a', 'b'): 1}

In [44]:
d

{('a', 'b'): 1}

In [45]:
hash(('a', 'b'))

-2513720613213709825

In [46]:
dt = datetime.datetime.now()

In [48]:
dt.timestamp()

1506220488.572322

In [49]:

def cache(exp=0):
    def _cache(fn):
        cache = {}
        @functools.wraps(fn)
        def wrap(*args, **kwargs):
            key = []

            # 对默认值进行 预处理
            # 需要把默认值  和 其参数 存储起来
            # names 中存储的是什么？是我们的参数key
            # 默认值不需要存储，因为 可以通过v.default
            names = set()

            # 处理位置参数
            params = inspect.signature(fn).parameters
            for i, arg in enumerate(args):
                name = list(params.keys())[i]
                key.append((name, arg))

                names.add(name)

            # 关键字参数
            # for k, v in kwargs.items():
            #    key.append((k, v))

            key.extend(kwargs.items())
            names.update(kwargs.keys())

            # 如果 参数中 有默认值，就需要 往key中添加一对 k-v 对
            for k, v in params.items():
                if k not in names:
                    key.append((k, v.default))

            key.sort(key=lambda x: x[0])

            key = '&'.join(['{}={}'.format(name, arg) for name, arg in key])

            print(key)


            # TODO key 如何瓶装
            if key in cache.keys():
                # TODO 超时检测
                # key -> 'x=1&y=3&timestamp=12345678'
                ret, timestamp = cache[key]
                if exp == 0 or datetime.datetime.now().timestamp() - timestamp < exp:

                    print('cache hit')
                    return cache[key]
            ret = fn(*args, **kwargs)
            cache[key] = (ret, datetime.datetime.now().timestamp())
            print('cache miss')
            return ret
        return wrap
    return _cache

In [50]:
@cache(5)
def add(x, y=3):
    return x + y

In [54]:
add(1, 3)

x=1&y=3
cache miss


4

### 写一个命令分发器

- 程序员可以方便的注册函数到某个命令，当用户输入某个命令时，路由到注册的函数，如果此命令没有注册函数，则执行默认函数

register(command, fn)

run():

    while True:
    
        'exit'
        
        

In [ ]:
@command
def add(x, y):
    return x + y 

In [55]:
commands = {}

In [56]:
def register(command, fn):
    commands[command] = fn

In [57]:
def default_fn():
    print('undefined command')

In [58]:
def run():
    while True:
        cmd = input('>>')
        if cmd.strip() == 'quit':
            return
        
        commands.get(cmd.strip(), default_fn)()

In [61]:
d = {'a': default_fn}

In [62]:
d.get('a', 'b')()

undefined command


In [64]:
def commands():
    commands = {}
    
    def register(command, fn):
        commands[command] = fn
        
    def default_fn():
        print('undefined command')
        
    def run():
        while True:
            cmd = input('>>')
            if cmd.strip() == 'quit':
                return

            commands.get(cmd.strip(), default_fn)()
            
    return register, run

In [65]:
register, run = commands()

In [71]:
def pa():
    print('papapa')

In [73]:
register('papapa', pa)

In [74]:
run()

>>pa
undefined command
>>papapa
papapa
>>quit


In [81]:
def commands():
    commands = {}
    
    
    def register(command):
        def _register(fn):
            commands[command] = fn
            return fn
        return _register
        
    def default_fn():
        print('undefined command')
        
    def run():
        while True:
            cmd = input('>>')
            if cmd.strip() == 'quit':
                return

            commands.get(cmd.strip(), default_fn)()
            
    return register, run

In [86]:
register, run = commands()

In [90]:
@register('papa')
def papapa():
    print('pa')
    return 1

In [ ]:
run()

>>pa
undefined command
>>papa
pa


### 第五周作业

1. 实现base64解码（不需要用装饰器）

2. 带过期的LRU缓存装饰器（要用装饰期）